# Shapley Value Calculation
A popular approach to tackle the problem of XAI is to use concepts from game theory in particular cooperative game theory.
The most popular method is to use the **Shapley Values** named after Lloyd Shapley, who introduced it in 1951 with his work *"II: The Value of an n-Person Game"*.

## Cooperative Game Theory
Cooperative game theory deals with the study of games in which players/participants can form groups to achieve a collective payoff. More formally a cooperative game is defined as a tuple $(N,\nu)$ where:
- $N$ is a finite set of players
- $\nu$ is a characteristic function that maps every coalition of players to a real number, i.e. $\nu:2^N \rightarrow \mathbb{R}$

Of particular interest is to find a concept that distributes the payoff of $\nu(N)$ among the players, as it is assumed that the *grand coalition* $N$ is formed.
The distribution of the payoff among the players is called a *solution concept*.

## Shapley Values: A Unique Solution Concept
Given a cooperative game $(N,\nu)$, the Shapley value is a payoff vector dividing the total payoff $\nu(N)$ among the players. The Shapley value of player $i$ is denoted by $\phi_i(\nu)$ and is defined as:
$$
\phi_i(\nu) := \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N|-|S|-1)!}{|N|!} [\nu(S \cup \{i\}) - \nu(S)]
$$
and can be interpreted as the  average marginal contribution of player $i$ across all possible permutations of the players.
Its popularity arises from uniquely satisfies the following properties:
- **Efficiency**: The sum of the Shapley values equals the total payoff, i.e. $\sum_{i \in N} \phi_i(\nu) = \nu(N)$
- **Symmetry**: If two players $i$ and $j$ are such that for all coalitions $S \subseteq N \setminus \{i,j\}$, $\nu(S \cup \{i\}) = \nu(S \cup \{j\})$, then $\phi_i(\nu) = \phi_j(\nu)$
- **Additivity**: For a game $(N,\nu + \mu)$ based on two games $(N,\nu)$ and $(N,\mu)$, the Shapley value of the sum of the games is the sum of the Shapley values, i.e. $\phi_i(\nu + \mu) = \phi_i(\nu) + \phi_i(\mu)$
- **Dummy Player**: If for a player $i$ is holds for all coalitions $S \subseteq N \setminus \{i\}$, $\nu(S \cup \{i\}) - \nu(S) = \nu(\{i\})$ then $\phi_i(\nu) = \nu(\{i\})$

## Shapley Values: Cooking Game
To illustrate the concept of Shapley values, we consider a simple example of a cooking game.
 The game consists of three players(cooks), Alice, Bob, and Charlie, who are cooking a meal together.
 The characteristic function $\nu$ maps each coalition of players to the quality of the meal.